# Homework 08
This homework is based on the clustering lectures. Check the lecture notes and TA notes - they should help!

## Question 1
This question will walk you through creating your own `kmeans` function.

#### a) What are the steps of `kmeans`?
**Hint**: There are 4 steps/builder functions that you'll need.

1. Assign each point to a cluster at random.
2. Calculate the mean position of each cluster using the previous assignments.
3. Loop through the points and assign each point to the cluster to whose center it is closest.
4. Repeat this process until the centers stop moving around.

#### b) Create the builder function for step 1.

In [143]:
library(dplyr)

In [144]:
assign_cluster <- function(n, k) {
  repeat {
    clusters <- sample.int(k, n, replace = TRUE)
    if (length(unique(clusters)) == k) break
  }
  clusters
}

#### c) Create the builder function for step 2.

In [145]:
calculate_mean <- function(df) {
  feature_cols <- setdiff(names(df), "cluster")
  colnames(df)[colnames(df) %in% feature_cols] <- paste0("V", seq_along(feature_cols))
  
  df %>%
    group_by(cluster) %>%
    summarise(across(starts_with("V"), mean), .groups = "drop") %>%
    arrange(cluster) %>%
    mutate(label = paste0("μ", row_number() - 1))
}

#### d) Create the builder function for step 3.
*Hint*: There are two ways to do this part - one is significantly more efficient than the other. You can do either.  

In [146]:
assign_to_nearest <- function(df_points, centers_df) {
  C <- as.matrix(centers_df[, !names(centers_df) %in% c("cluster", "label"), drop = FALSE])
  X <- as.matrix(df_points)
  dist_matrix <- outer(rowSums(X^2), rowSums(C^2), "+") - 2 * (X %*% t(C))
  cluster <- max.col(-dist_matrix)
  df_out <- as.data.frame(X)
  df_out$cluster <- cluster
  df_out
}

#### e) Create the builder function for step 4.

In [147]:
converged <- function(prev_labels, curr_labels) {
  identical(prev_labels, curr_labels)
}

#### f) Combine them all into your own `kmeans` function.

In [148]:
kmeans_custom <- function(df, k) {
  df$cluster <- assign_cluster(nrow(df), k)
  
  repeat {
    old_clusters <- df$cluster
    centers_df <- calculate_mean(df)
    df <- assign_to_nearest(df %>% select(-cluster), centers_df)
    
    if (all(df$cluster == old_clusters)) {
      break
    }
  }
  
  list(labels = df$cluster, means = centers_df)
}


## Question 2
This is when we'll test your `kmeans` function.
#### a) Read in the `voltages_df.csv` data set. 

In [149]:
voltages_df <- read.csv("voltages_df.csv")

#### b) Call your `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$means`. 

In [150]:
results <- kmeans_custom(voltages_df, k = 3)


print(results$labels)
print(as.matrix(results$means))



  [1] 2 1 1 1 1 1 3 3 1 2 2 2 3 2 1 2 2 2 2 2 1 1 1 1 1 2 2 2 3 3 3 3 1 3 3 1 1
 [38] 2 3 1 2 2 2 3 1 3 3 1 3 3 1 3 2 2 3 2 1 2 3 1 2 2 1 2 3 3 1 3 2 3 2 1 3 3
 [75] 1 1 2 3 3 3 3 3 1 3 3 1 1 1 2 3 3 3 1 3 1 1 3 1 1 3 3 3 1 3 2 2 3 2 2 1 2
[112] 2 1 3 2 3 2 2 1 1 1 1 2 2 3 1 1 3 2 3 1 2 1 3 2 1 1 1 2 3 2 1 1 3 2 2 1 2
[149] 2 3 2 3 1 1 3 2 3 3 1 2 3 3 2 1 2 3 1 2 1 2 2 3 1 2 3 3 1 3 2 1 3 1 3 2 3
[186] 1 1 2 1 2 2 1 2 2 3 3 2 1 2 2 3 2 3 3 1 2 1 3 1 2 1 3 2 3 3 1 3 3 1 1 1 1
[223] 1 1 1 3 3 3 3 2 1 2 3 1 1 3 3 2 2 3 3 2 3 3 1 3 2 3 3 1 3 1 2 3 1 3 3 2 2
[260] 2 1 1 3 2 1 1 1 3 3 3 2 1 1 1 3 2 3 3 1 1 3 2 3 2 2 1 3 1 1 3 3 2 3 3 2 1
[297] 2 3 3 1 1 3 2 2 2 3 2 1 1 1 2 2 2 2 2 2 1 2 1 2 2 3 1 3 2 2 2 2 1 2 1 2 3
[334] 2 1 1 3 3 2 1 3 1 3 3 2 3 1 3 2 2 3 1 1 1 2 3 1 2 3 3 1 1 1 1 2 3 2 1 2 2
[371] 2 2 2 1 1 3 3 2 2 2 2 3 3 2 2 3 3 1 1 3 2 1 1 1 1 3 1 1 3 2 2 2 2 1 1 2 2
[408] 3 3 3 1 3 3 2 1 1 1 3 1 1 1 2 2 1 2 3 1 1 1 1 3 3 1 3 2 1 2 2 2 3 3 3 3 2
[445] 1 2 2 2 3 1 2 2 2 1 1 2 1 1 2 3 1 

#### c) Call R's `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$cluster`. 
*Hint*: Use the `as.matrix()` function to make the `voltages_df` data frame a matrix before calling `kmeans()`.

In [151]:
voltages_matrix <- as.matrix(voltages_df)
results <- kmeans(voltages_matrix, centers = 3)
print(results$cluster) 
print(results$centers) 

  [1] 3 1 1 1 1 1 2 2 1 3 3 3 2 3 1 3 3 3 3 3 1 1 1 1 1 3 3 3 2 2 2 2 1 2 2 1 1
 [38] 3 2 1 3 3 3 2 1 2 2 1 2 2 1 2 3 3 2 3 1 3 2 1 3 3 1 3 2 2 1 2 3 2 3 1 2 2
 [75] 1 1 3 2 2 2 2 2 1 2 2 1 1 1 3 2 2 2 1 2 1 1 2 1 1 2 2 2 1 2 3 3 2 3 3 1 3
[112] 3 1 2 3 2 3 3 1 1 1 1 3 3 2 1 1 2 3 2 1 3 1 2 3 1 1 1 3 2 3 1 1 2 3 3 1 3
[149] 3 2 3 2 1 1 2 3 2 2 1 3 2 2 3 1 3 2 1 3 1 3 3 2 1 3 2 2 1 2 3 1 2 1 2 3 2
[186] 1 1 3 1 3 3 1 3 3 2 2 3 1 3 3 2 3 2 2 1 3 1 2 1 3 1 2 3 2 2 1 2 2 1 1 1 1
[223] 1 1 1 2 2 2 2 3 1 3 2 1 1 2 2 3 3 2 2 3 2 2 1 2 3 2 2 1 2 1 3 2 1 2 2 3 3
[260] 3 1 1 2 3 1 1 1 2 2 2 3 1 1 1 2 3 2 2 1 1 2 3 2 3 3 1 2 1 1 2 2 3 2 2 3 1
[297] 3 2 2 1 1 2 3 3 3 2 3 1 1 1 3 3 3 3 3 3 1 3 1 3 3 2 1 2 3 3 3 3 1 3 1 3 2
[334] 3 1 1 2 2 3 1 2 1 2 2 3 2 1 2 3 3 2 1 1 1 3 2 1 3 2 2 1 1 1 1 3 2 3 1 3 3
[371] 3 3 3 1 1 2 2 3 3 3 3 2 2 3 3 2 2 1 1 2 3 1 1 1 1 2 1 1 2 3 3 3 3 1 1 3 3
[408] 2 2 2 1 2 2 3 1 1 1 2 1 1 1 3 3 1 3 2 1 1 1 1 2 2 1 2 3 1 3 3 3 2 2 2 2 3
[445] 1 3 3 3 2 1 3 3 3 1 1 3 1 1 3 2 1 

#### d) Are your labels/clusters the same? If not, why? Are your means the same?

The cluster labels in my implementation do not match R’s kmeans() output numerically, but this is expected because K-means cluster numbering is arbitrary and can vary depending on random initialization. For example, what R labels as Cluster 2 (with a centroid around V2 ≈ 1.31 and V3 ≈ 1.16) corresponds closely to my Cluster 1, which shows nearly identical mean values across those same variables. Likewise, R’s Cluster 3 (V2 ≈ 0.94, V3 ≈ 0.76) matches my Cluster 3, and R’s Cluster 1 (V2 ≈ 1.09, V3 ≈ 0.93) aligns with my Cluster 2.

Although the numeric labels differ, the centroid patterns are highly consistent across both implementations. Each cluster shares the same initial mean at V1 ≈ –1.03, showing that the data were scaled similarly in both cases. The relative magnitudes across early variables (V2 through V5) are also nearly identical, showing that both algorithms converged to the same underlying partition of the data. 

Overall, the two algorithms produced identical cluster means but with different cluster labels.

## Question 3
#### a) Explain the process of using a for loop to assign clusters for kmeans.

In the for-loop version, each data point is handled one at a time. For every point, you loop through all cluster centroids, compute the distance between the point and each centroid, and assign the point to the cluster with the smallest distance. This creates a nested loop (over points and clusters) and explicitly repeats distance calculations for each pair.

#### b) Explain the process of vectorizing the code to assign clusters for kmeans.

In the vectorized version, instead of looping, matrix operations are used to calculate all point–cluster distances at once. Two large replicated matrices are constructed: one where every data point is repeated k times (once for each cluster), and another where every cluster mean is repeated n times (once for each data point). These matrices now have the same dimensions, allowing direct subtraction to compute all pairwise distances in a single operation.

#### c) State which (for loops or vectorizing) is more efficient and why.

Vectorizing is far more efficient. R is optimized for vector and matrix operations and can process large numeric arrays simultaneously. In contrast, when using for loops, R must execute each element one at a time which is much slower.

## Question 4
#### When does `kmeans` fail? What assumption does `kmeans` use that causes it to fail in this situation?

K-means fails when clusters are not spherical, equally sized, or well separated. It assumes data are distributed as uniformly shaped Gaussian blobs, so when clusters are elongated, curved, or vary in density, the centroid-based distance measure misclassifies points and produces incorrect boundaries.

## Question 5
#### What assumption do Guassian mixture models make?

Gaussian Mixture Models assume that the data come from a mixture of several Gaussian distributions, each with its own mean and covariance. This means each cluster is modeled as a separate Gaussian, allowing clusters to differ in size, shape, and orientation instead of being uniform like in K-means.

## Question 6
#### What assumption does spectral clustering make? Why does this help us?

Spectral clustering assumes that points are more likely to belong to the same cluster if they are close to one another based on a similarity measure. This helps because it removes the strong geometric assumptions made by algorithms like K-means. It only requires a notion of pairwise similarity, not that clusters be spherical or evenly sized. As a result, spectral clustering can identify complex, non-linear cluster shapes that K-means would fail to capture.

## Question 7
#### Define the gap statistic method. What do we use it for?

The gap statistic method helps determine the best number of clusters in a dataset. It compares the within-cluster variation for each value of k to that from a reference dataset with randomly distributed points. The difference between these two values, called the “gap,” measures how much better the real clustering is than random chance. The optimal number of clusters is chosen where this gap is largest, showing the most meaningful separation of the data.